In [13]:
import pandas as pd
import pprint

In [2]:
lyrics = pd.read_csv('../genius_lyrics.csv')

In [9]:
text_corpus = lyrics['lyrics'].tolist()

In [77]:
# Create a set of frequent words
stoplist = set('for a of the and to in'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in text_corpus]

# Count word frequencies
from collections import defaultdict

frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1


# Only keep words that appear more than once
processed_corpus = [[token for token in text if frequency[token] > 1] for text in texts]



AttributeError: 'str' object has no attribute 'to_csv'

In [20]:
from gensim import corpora

dictionary = corpora.Dictionary(processed_corpus)
corpus = [dictionary.doc2bow(text) for text in texts]

In [21]:
from gensim import models

tfidf = models.TfidfModel(corpus)  # step 1 -- initialize a model

In [22]:
corpus_tfidf = tfidf[corpus]


Latent Semantic Modeling

In [26]:
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=4)  # initialize an LSI transformation
corpus_lsi = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [28]:
lsi_model.print_topics(4) #profanity seems to follow other profanity

[(0,
  '-0.118*"love" + -0.110*"she" + -0.098*"oh," + -0.097*"you," + -0.096*"oh" + -0.094*"we" + -0.094*"you\'re" + -0.091*"baby," + -0.089*"wanna" + -0.088*"her"'),
 (1,
  '0.930*"-" + 0.266*"feat." + 0.141*"&" + 0.060*"lil" + 0.032*"drake" + 0.027*"(ft." + 0.023*"*" + 0.022*"ey" + 0.022*"(for" + 0.021*"ft."'),
 (2,
  '0.938*"na" + 0.154*"na," + 0.086*"la" + 0.076*"oh" + 0.055*"beautiful," + 0.050*"beautiful" + -0.049*"she" + 0.045*"what?" + 0.043*"tonight" + -0.039*"her"'),
 (3,
  '-0.243*"she" + 0.193*"oh" + -0.158*"her" + -0.142*"nigga" + -0.140*"niggas" + -0.137*"na" + 0.135*"oh," + -0.133*"fuck" + -0.105*"ayy" + -0.104*"they"')]

Similarity queries

In [30]:
from gensim import similarities
index = similarities.MatrixSimilarity(corpus_lsi) #transform corpus to LSI space and index it

In [88]:
from pprint import pprint as print
from gensim.models.fasttext import FastText
import os

#preparing text for Fast Text Modeling
corpus_to_save = ' '.join([' '.join(processed_corpus[i]) for i in range(len(processed_corpus))])

#Filepath to pass into the model:
with open("../yearendcorpus.txt", "w", encoding='utf-8') as output:
    output.write(str(corpus_to_save))
    
corpus_file = '../yearendcorpus.txt'


model = FastText(vector_size=100)

# build the vocabulary
model.build_vocab(corpus_file=corpus_file)

# train the model
model.train(
    corpus_file=corpus_file, epochs=model.epochs,
    total_examples=model.corpus_count, total_words=model.corpus_total_words,
)

print(model)